In [1]:
import pandas as pd
import numpy as np
import nltk 
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
tqdm.pandas()
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

/Users/chenjiqing/anaconda3/envs/Class/lib/python3.7/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
import nltk
nltk.download('words')
nltk.download('stopwords')
words = nltk.corpus.words.words()
sw = stopwords.words('english')

[nltk_data] Downloading package words to
[nltk_data]     /Users/chenjiqing/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenjiqing/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### load review data, clean the reviews, and save as .csv

In [4]:
df = pd.read_json('/Users/chenjiqing/Public/2020_Fall_term/data/Toys_and_Games_Reviews_training.json', lines=True)

In [5]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"08 12, 2012",BDF1A2B47461BE6FD7B8AD896005E061,5FC455C18F7DD81A071E51B6B60A72BD,73D759DCDF02B4DDBD3FB7D29E88C6C3,I always look for this brand of cards because ...,durable cards,1344729600,NaN,NaN,NaN
1,5,False,"01 24, 2011",91BDB36705C359620E8E0959DEFCFC79,5FC455C18F7DD81A071E51B6B60A72BD,80C0DC9C8FAB1C9BE0E1D5E83965360C,"Other than KEM's or DaVinchi brands, these car...",some of the best Cards,1295827200,4,NaN,NaN
2,5,True,"11 18, 2016",B14C8E6A493C844D2AEDCC61207BB6B1,5FC455C18F7DD81A071E51B6B60A72BD,C133754F62F0EB10D1FFF543786622DC,The better ive ever seen!!!! Excelent!,Five Stars,1479427200,NaN,NaN,NaN
3,5,True,"03 7, 2016",D53FA93ED4A6B67662D920473DE9C7F7,5FC455C18F7DD81A071E51B6B60A72BD,59820C0C923F245A167B53E24FFBF5BB,get it,Five Stars,1457308800,NaN,NaN,NaN
4,5,False,"05 22, 2015",B59D69D00FEA8B83388D292010CAB2E9,5FC455C18F7DD81A071E51B6B60A72BD,2843F6C4145178E4F87A6CD895F4887F,Really nice cards. Will make great use of them...,playing cards,1432252800,NaN,NaN,NaN


In [7]:
print("Number of samples: {} Number of Features: {}".format(df.shape[0], df.shape[1]))

Number of samples: 1460818 Number of Features: 12


In [14]:
clean_df = df[df['reviewText'].notna()]
df2 = clean_df[['asin', 'overall', 'reviewText', 'summary']]

In [15]:
avg = df2[['asin', 'overall']].groupby('asin').mean()
# assian traget=1 if averaged overall>4.5 and target=0 otherwise
avg2 = avg.assign(
    Target = lambda dataframe: dataframe['overall'].map(lambda overall: "1" if overall >4.5 else "0") 
)
avg2 = avg2.rename_axis("asin").reset_index()
avg2.columns = ['asin','overall_avg', 'Target']

# select coloumns
df3 = df2[['asin', 'overall', 'reviewText', 'summary']]

df4 = df3.groupby('asin')['reviewText'].apply(' '.join).reset_index()
df4.head()
df4.shape

(63017, 2)

In [53]:
# assign target to df2
dft = df4.merge(avg2, on = 'asin', how ='left')

,asin,reviewText,overall_avg,Target
23,001046FB00BA52AC83EC1AFA36639395,Could not keep balls inflated after blowing th...,4.000000,0
24,0010BA5BFC56BAAB9AFCC9662AEF5D71,Wow! This is the real deal. It is fully as adv...,4.700000,1
25,001284527CE20A55C595B9C0CFEC7568,"My 4 year old loves these Grandsons are five, ...",4.888889,1
26,0013DF468664DE1FA3CD41A72E671C9D,"Great jigsaw puzzle! Challenging in places, bu...",4.666667,1
27,0014D30399BE488500B05894FD3186A7,My 7 month old grandson loves this. The only ...,4.571429,1
28,001C05706A0288190C0AF0455310E657,My son loves this toy. It is easier to deal wi...,4.772727,1
29,001D1C8B0D953E22EE70BDFE53D966FC,I got them exactly the day that they were expe...,4.400000,0
30,001DBBD8771DFFEDD764111BA02203EC,Son loved them My children love it! My son lov...,4.600000,1
31,001DC2730F3CB75EEAFDBA036CF4A268,"When these arrived, one already didn't light u...",3.181818,0


In [3]:
# dft.to_csv('df_combined_text.csv')
dft = pd.read_csv('df_combined_text.csv')
dft

,Unnamed: 0,Unnamed: 0.1,asin,reviewText,overall_avg,Target,count,sentiment_scores
0,0,0,000254CDA01E5AE839E7EE3F6FC1D2F1,The child who recieved them opened the box and...,4.625000,1,24,0.487121
1,1,1,0002E81AED1B9840434EE28E39876BE4,Great gift for my special needs cousin Blocks ...,2.538462,0,13,-0.223308
2,2,2,0003806D9BE7957D8AB9AF9EC04F52D0,I needed something to occupy kids indoors that...,5.000000,1,6,0.464500
3,3,3,00049B538C925B3EF702FFF6FBD48DFE,This is a very nicely detailed and painted die...,4.625000,1,8,0.133913
4,4,4,0004D0377FDC88F2DD349170E36324E7,Cute This bear is so cool and cute. The fabric...,5.000000,1,5,0.557400
...,...,...,...,...,...,...,...,...
63012,63012,63012,FFFAB16CD5964F60F014EFAA1210AFEE,These stickers are really adorable. The variet...,4.928571,1,14,0.517586
63013,63013,63013,FFFAB94C16EAEC9E1DEBF24E231C53BB,"great little toy. <div id=""video-block-R3SMI4X...",4.380952,0,21,0.530857
63014,63014,63014,FFFB0C64CBC232115CE8FFF2BE870122,I am a collector of everything Firefly / Seren...,4.850000,1,20,0.529530
63015,63015,63015,FFFB5BB91B42C9A34457AD075B499DF9,"This made a cute Easter gift, but was soon for...",4.090909,0,11,0.557400


In [4]:
dft_small = dft.sample(n =10000)
print(dft_small.Target.value_counts())

1    5461
0    4539
Name: Target, dtype: int64


# Bag of Words

In [19]:
# create a count vectorizer object
token = RegexpTokenizer(r'[a-zA-Z]+')
count_vectorizer = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)

# fit the count vectorizer using the text data
text_counts = count_vectorizer.fit_transform(dft['reviewText'])

# collect the vocabulary items used in the vectorizer
# dictionary = count_vectorizer.vocabulary_.items()

# words appear in documents.
tokens = count_vectorizer.get_feature_names()
#print(tokens)

In [20]:
# split traning data and testing data
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dft['Target'], test_size=0.2, random_state=1)

### Bag of Words-multinomial NB

In [27]:
# from sklearn.model_selection import cross_val_score ## for cross-validation
from sklearn import metrics

clf = MultinomialNB().fit(X_train, y_train)

predicted= clf.predict(X_test)

print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))
from sklearn.metrics import classification_report
print(classification_report(y_test,predicted))

# scores = cross_val_score(clf, text_counts, dft['Target'], cv=5) ## for cross-validation
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) ## for cross-validation

MultinomialNB Accuracy: 0.7145350682323072
              precision    recall  f1-score   support

           0       0.72      0.61      0.66      5774
           1       0.71      0.80      0.75      6830

    accuracy                           0.71     12604
   macro avg       0.72      0.71      0.71     12604
weighted avg       0.72      0.71      0.71     12604



### Bag of Words- logisticRegression

In [29]:
#Import model
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn import metrics
#Create a Classifier
clf2 = LogisticRegression(C=1, penalty='l1', solver='liblinear') # Linear Kernel

#Train the model using the training sets
clf2.fit(X_train, y_train)

#Predict the response for test dataset
y_pred2 = clf2.predict(X_test)
# print(y_pred)
print("LASSO-LR Accuracy:",metrics.accuracy_score(y_test, y_pred2))
# print(clf.decision_function(X_test))
# print(clf.predict_proba(X_test))

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

LASSO-LR Accuracy: 0.7711044112980007
0.7711044112980007
              precision    recall  f1-score   support

           0       0.76      0.73      0.75      5774
           1       0.78      0.81      0.79      6830

    accuracy                           0.77     12604
   macro avg       0.77      0.77      0.77     12604
weighted avg       0.77      0.77      0.77     12604



### Bag of Words- logisticRegression-CrossValidation

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics

#Create a svm Classifier
clf3 = LogisticRegressionCV(penalty='l1', solver='liblinear',cv=10) # Linear Kernel

#Train the model using the training sets
clf3.fit(X_train, y_train)
#Predict the response for test dataset
y_pred3 = clf3.predict(X_test)
# print(y_pred2)
print("logisticRegression-cross-validation Accuracy:",metrics.accuracy_score(y_test, y_pred3))
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred3))

# TFIDF

In [5]:
tf=TfidfVectorizer()
text_tf= tf.fit_transform(dft_small['reviewText'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    text_tf, dft_small['Target'], test_size=0.2, random_state=1)

# SMOTE

In [7]:
# pip install imblearn
# check version number
import imblearn
print(imblearn.__version__)
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__)) # The scikit-learn version should be 0.23.2.

0.7.0
The scikit-learn version is 0.23.2.


## SMOTE-TFIDF-LogisticRegression (default k = 5)

In [6]:
#Import model
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn import metrics
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

model = LogisticRegression(C=1, penalty='l1', solver='liblinear') # Linear Kernel
oversample = SMOTE()
X, y = oversample.fit_resample(X_train, y_train)

#Train the model using the training sets
model.fit(X, y)

#Predict the response for test dataset
y_pred4 = model.predict(X_test)

# evaluate 
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      5774
           1       0.83      0.82      0.82      6830

    accuracy                           0.81     12604
   macro avg       0.81      0.81      0.81     12604
weighted avg       0.81      0.81      0.81     12604



## SMOTE-TFIDF-LogisticRegression (k = 1~7)

In [8]:
k_values = [1,2,3,4,5,6,7]
for k in k_values:
    model = LogisticRegression(C=1, penalty='l1', solver='liblinear') # Linear Kernel
    oversample = SMOTE(k_neighbors=k)
    X, y = oversample.fit_resample(X_train, y_train)
    model.fit(X, y)
    y_pred5 = model.predict(X_test)
    print('k = ', k)
    print(classification_report(y_test,y_pred5))

k =  1
              precision    recall  f1-score   support

           0       0.79      0.80      0.80      5774
           1       0.83      0.82      0.82      6830

    accuracy                           0.81     12604
   macro avg       0.81      0.81      0.81     12604
weighted avg       0.81      0.81      0.81     12604

k =  2
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      5774
           1       0.83      0.82      0.83      6830

    accuracy                           0.81     12604
   macro avg       0.81      0.81      0.81     12604
weighted avg       0.81      0.81      0.81     12604

k =  3
              precision    recall  f1-score   support

           0       0.79      0.80      0.80      5774
           1       0.83      0.82      0.82      6830

    accuracy                           0.81     12604
   macro avg       0.81      0.81      0.81     12604
weighted avg       0.81      0.81      0.81     12604

### TFIDF-multinomial NB

In [22]:
clf = MultinomialNB().fit(X_train, y_train)

predicted= clf.predict(X_test)

print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

MultinomialNB Accuracy: 0.7159631862900666
              precision    recall  f1-score   support

           0       0.74      0.59      0.65      5774
           1       0.70      0.83      0.76      6830

    accuracy                           0.72     12604
   macro avg       0.72      0.71      0.71     12604
weighted avg       0.72      0.72      0.71     12604



### TFIDF-logistic regression (for comparison with TFIDF-Late Fusion, I only use 10,000 samples)

In [8]:
#Import model
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn import metrics
#Create a Classifier
clf3 = LogisticRegression(C=1, penalty='l1', solver='liblinear') # Linear Kernel

#Train the model using the training sets
clf3.fit(X_train, y_train)

#Predict the response for test dataset
y_pred3 = clf3.predict(X_test)
# print(y_pred)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, y_pred3))
# print(clf.decision_function(X_test))
# print(clf.predict_proba(X_test))

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred3))

Logistic Regression Accuracy: 0.789
              precision    recall  f1-score   support

           0       0.80      0.75      0.77       953
           1       0.78      0.83      0.80      1047

    accuracy                           0.79      2000
   macro avg       0.79      0.79      0.79      2000
weighted avg       0.79      0.79      0.79      2000



## TFIDF-Late Fusion (only use 10,000 samples; memmory isn't enough for all samples)

In [9]:
from sklearn.ensemble import VotingClassifier
clf1 = LogisticRegression(C=1, penalty='l1', solver='liblinear')
clf2 = GaussianNB()
clf3 = RandomForestClassifier(n_estimators=50, random_state=1)
eclf = VotingClassifier(estimators=[('lr', clf1),('rf', clf3), ('gnb', clf2)],voting='hard')
eclf.fit(X_train.toarray(), y_train)
y_pred_V = eclf.predict(X_test.toarray())
print('Late Fusion- LR,GNB,RF')
print(classification_report(y_test,y_pred_V))

Late Fusion- LR,GNB,RF
              precision    recall  f1-score   support

           0       0.77      0.69      0.73       953
           1       0.74      0.82      0.78      1047

    accuracy                           0.76      2000
   macro avg       0.76      0.75      0.75      2000
weighted avg       0.76      0.76      0.75      2000



In [13]:
y_pred_V

array([0, 1, 1, ..., 0, 0, 0])

## SMOTE - Late Fusion with GridSearchCV (only use 10,000 samples; memmory isn't enough for all samples)

In [11]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X_train, y_train)
params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200]}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X.toarray(), y)
y_pred_V = grid.predict(X_test.toarray())
print(classification_report(y_test,y_pred_V))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73       953
           1       0.75      0.77      0.76      1047

    accuracy                           0.75      2000
   macro avg       0.75      0.74      0.74      2000
weighted avg       0.75      0.75      0.75      2000



### Prediction csv (not includes SMOTE / Late Fusion / GridSearch / cross-validation)

In [7]:
toy_test = pd.read_json('/Users/chenjiqing/Public/2020_Fall_term/data/Toys_and_Games_Reviews_test.json', lines=True)

In [8]:
toy_test.head()

,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,True,"01 9, 2014",F65C6B80659643241B55D2F8606CA762,B53842B7FB95A7EED604789C4527FED4,C13E13DA2073260C2194C15D782E86A9,I bought this item for my granddaughter and sh...,xmas present,1389225600,NaN,NaN,NaN
1,True,"01 7, 2014",B59B4F985BFB64754C6DA13A1825344C,B53842B7FB95A7EED604789C4527FED4,EB6C1EA748A0926CF57E2915043A1B2C,My kids didn't care for this. It's not exactly...,"Not what I expected, my kids didn't care for it.",1389052800,NaN,NaN,NaN
2,True,"06 17, 2014",7CA53EA426A3925851477B594C14C1B1,B53842B7FB95A7EED604789C4527FED4,3C05450791D51827DF4ADAC95DB284C3,"Don't waste time or money on this, it does not...",Does not work,1402963200,NaN,NaN,NaN
3,True,"03 26, 2014",2421E7DA831836AC70B3599A07CC6813,B53842B7FB95A7EED604789C4527FED4,EFB1465497243F7027F6D6C101F3CDDC,This is a fun idea but unfortunately the handl...,Fun idea,1395792000,NaN,NaN,NaN
4,False,"02 22, 2014",3A27A6CCDFEB41598C949DEE4B130773,B53842B7FB95A7EED604789C4527FED4,3CF804E7182AB12879C33C914E1C5CD8,This is a very sturdy tub toy and has handled ...,Super toy and very durable.,1393027200,NaN,NaN,NaN


In [9]:
# select coloumns
toy_test2 = toy_test[['asin', 'reviewText']]
clean_toy_test = toy_test2[toy_test2['reviewText'].notna()]
print(clean_toy_test.shape)
clean_toy_test2 = clean_toy_test.groupby('asin')['reviewText'].apply(' '.join).reset_index()
clean_toy_test2.head()
print(clean_toy_test2.shape)
print(clean_toy_test2.head())

(367913, 2)
(15755, 2)
                               asin  \
0  00027761289A98A2884B5F152D3DBF91   
1  000D550D33ABEA8BAF3C8E0FD88C1707   
2  0024011FC5EBE2E578E9032EC2EEE3EA   
3  00255AA8E14A573176D24AC68524541E   
4  00290349FBB97F31976D0471975A4789   

                                          reviewText  
0  Bought for a contest. Although for a child it ...  
1  I do a lot of model kit building, mainly warsh...  
2  I bought this game for the kids at my son's 2n...  
3  I bought this as a gift and the recipient abso...  
4  Daughter likes this very much. Cheap alternati...  


In [10]:
tf2=TfidfVectorizer(vocabulary = tf.vocabulary_)
text_tf2= tf2.fit_transform(clean_toy_test2['reviewText'])
print(text_tf2)

  (0, 151962)	0.023741331568156925
  (0, 150233)	0.04345880328538978
  (0, 149852)	0.09747095777753212
  (0, 149579)	0.06048949819576235
  (0, 149535)	0.06183491648218076
  (0, 149032)	0.2778927754827474
  (0, 149007)	0.4872246723760914
  (0, 148765)	0.051447773837739984
  (0, 148460)	0.027844353062162683
  (0, 148136)	0.06312839290438638
  (0, 148060)	0.025540837543542732
  (0, 147699)	0.02744615601266229
  (0, 147326)	0.04353980785843322
  (0, 144443)	0.03814688138154437
  (0, 144436)	0.06730124544369397
  (0, 143947)	0.05292031195780934
  (0, 138762)	0.044955461570473816
  (0, 138179)	0.05761758813169907
  (0, 136830)	0.01981248823203037
  (0, 136132)	0.2620215065391931
  (0, 134375)	0.06675042782912839
  (0, 133683)	0.15166240771362147
  (0, 130133)	0.07600260619822456
  (0, 128897)	0.09791091504848393
  (0, 126167)	0.03350106024236067
  :	:
  (15754, 32708)	0.043311182308391996
  (15754, 32548)	0.05560010630609238
  (15754, 31284)	0.11200288028675763
  (15754, 31268)	0.04924598118

In [11]:
y_toy_test_pred = clf3.predict(text_tf2)

In [14]:
print(y_toy_test_pred)

[0 0 0 ... 0 0 0]


In [21]:
dft_predict1 = clean_toy_test2[['asin']]
print(dft_predict1)
dft_predict1['preds'] = y_toy_test_pred
print(dft_predict1)
dft_predict1.to_csv('predictions.csv', index=False)

                                   asin
0      00027761289A98A2884B5F152D3DBF91
1      000D550D33ABEA8BAF3C8E0FD88C1707
2      0024011FC5EBE2E578E9032EC2EEE3EA
3      00255AA8E14A573176D24AC68524541E
4      00290349FBB97F31976D0471975A4789
...                                 ...
15750  FFE0E00A3198C981F461CA920E1BE2E0
15751  FFE23C8B29D211456D9774F49E277905
15752  FFF41906931B0E2371E123299500D8E5
15753  FFF574CC84FD43D875FF7257B3797BF3
15754  FFF970B0EBBEA7752C6BAEBE3F4361D8

[15755 rows x 1 columns]
                                   asin  preds
0      00027761289A98A2884B5F152D3DBF91      0
1      000D550D33ABEA8BAF3C8E0FD88C1707      0
2      0024011FC5EBE2E578E9032EC2EEE3EA      0
3      00255AA8E14A573176D24AC68524541E      1
4      00290349FBB97F31976D0471975A4789      1
...                                 ...    ...
15750  FFE0E00A3198C981F461CA920E1BE2E0      0
15751  FFE23C8B29D211456D9774F49E277905      1
15752  FFF41906931B0E2371E123299500D8E5      0
15753  FFF574CC84FD43D87